##Protocol for annotation of Nahant phage genomes

No more waiting for other folks to get pipelines up and running... let's figure this out.

>UBLAST of prodigal-identified proteins against several sequence databases:
>>COG

>>PFam

>>ACLAME

>>Camera Viral Proteins (CVP)

>>KEGG

>>TARA Oceans Database

>UBLAST run via run_ublasts.py script in ./submissions; implemented on server using ./submissions/ublast_runs.slurm

In [87]:
#Functions to run annotation-associated programs:

import subprocess
import os

def run_prodigal_phage(inputfasta, out_gene, out_prot):
    to_run="prodigal -i "+inputfasta+" -o "+out_gene+" -a "+out_prot+" -p meta"
    subprocess.call(to_run.split(" "))
    
def run_formatudb(fastafile, databasefile="db.udb", ublast_path="/home/sbiller/usearch7.0.1090_i86linux64"):
    to_run=ublast_path+" -makeudb_ublast "+fastafile+" -output "+databasefile
    subprocess.call(to_run.split(" "))
    
def run_ublastp(fastafile, out_file, udb, evalue, ublast_path="/home/sbiller/usearch7.0.1090_i86linux64"):
    to_run=ublast_path+" -ublast "+fastafile+" -db "+udb+" -evalue "+evalue+" -accel 0.5 -blast6out "+out_file+" -top_hit_only"
    subprocess.call(to_run.split(" "))
    
def run_trna_scan(input_file, output):
    if os.path.exists(output):
        os.remove(output)
    args=["tRNAscan-SE", "-o", output, "-G", "-D","-N", input_file]
    subprocess.call(args)
    print("tRNA scan of "+input_file+" is done!")

def run_crt(path_to_crt, input_fasta, output):
    args="java -cp "+path_to_crt+" crt -minNR 2 "+input_fasta+" "+output
    subprocess.call(args.split(" "))

##Combining annotation outputs into one .gff3 file

Below: Creating DB dicts for quick access to blast DB sequence codes, annotations and OGs

In [5]:
import cPickle as pickle
import os
#create DB dicts for BLAST result processing, pickle for storage until needed:

#os.mkdir("./databases/pickled_dicts/")

#ACLAME:
aclame=open("./databases/DB_Info/aclame/aclame_proteins_all_0.4.tab").readlines()
aclame_dict={}

for line in aclame[2:-1]:
    protein=line.split("\t")[0]
    ncbi_ann=line.split("\t")[2]
    aclame_dict[protein]=ncbi_ann

pickle.dump(aclame_dict, open("./databases/pickled_dicts/aclame_dict.p","wb"))    

##COG needs two dbs:
cogs=open("./databases/DB_Info/COG/cog2003-2014.csv").readlines()         #all COG sequences and COG groups
cogs2=open("./databases/DB_Info/COG/cognames2003-2014.tab").readlines()   #COG group definitions/functions

cog_dict={}
cog_defs={}

#dict from gi to COG:
for line in cogs:
    gi=line.split(",")[0]
    cog=line.split(",")[6]
    cog_dict[gi]=cog
#dict from COG to function definition:
for line in cogs2:
    cog=line.split("\t")[0]
    func=line.split("\t")[2]
    cog_defs[cog]=func
    
pickle.dump(cog_dict, open("./databases/pickled_dicts/cog_dict.p","wb"))
pickle.dump(cog_defs, open("./databases/pickled_dicts/cog_def.p","wb"))

##Pfam needs two DBs as well:
pfams=open("./databases/DB_Info/PFam/Pfam-A.titles.txt").readlines()   #ID all pfam sequences in BLAST db
pfams2=open("./databases/DB_Info/PFam/Pfam-A.clans.tsv").readlines()   #Matches IDs to "clans" with functions
pfam_dict={}
pfam_defs={}

#sequence to pfam:
for line in pfams:
    seq=line.split(" ")[0].replace(">","")
    pfam=line.split(" ")[2].split(";")[0]
    pfam_dict[seq]=pfam
#pfam to function:
for line in pfams2:
    pfam=line.split("\t")[0]
    function=line.split("\t")[4]
    pfam_defs[pfam]=function
    
pickle.dump(pfam_dict, open("./databases/pickled_dicts/pfam_dict.p","wb"))
pickle.dump(pfam_defs, open("./databases/pickled_dicts/pfam_def.p","wb"))
    
##CAMERA Viral Proteins annotations are complicated; extracting definitions from sequence titles:
cvp=open("./databases/DB_Info/CVP/CVP_titles.txt").readlines()

cvp_dict={}

headers=[]

for line in cvp:
    defs={}
    line=line.replace(">","")
    annotation=""
    info=line.split("/")[1:]
    ID=line.split("/")[0].replace(" ","")
    for i in info: 
        if "=" in i:
            defs[i.split("=")[0]]=i.split("=")[1]
    if "DESCRIPTION" in defs.keys():
        annotation=defs["DESCRIPTION"]
    elif "definition" in defs.keys():
        annotation=defs["definition"]
    
    cvp_dict[ID]=annotation.replace('"','')
    
pickle.dump(cvp_dict, open("./databases/pickled_dicts/cvp_dict.p","wb"))

In [6]:
aclame_dict=pickle.load(open("./databases/pickled_dicts/aclame_dict.p","rb"))
cog_dict=pickle.load(open("./databases/pickled_dicts/cog_dict.p","rb"))
cog_defs=pickle.load(open("./databases/pickled_dicts/cog_def.p","rb"))
pfam_dict=pickle.load(open("./databases/pickled_dicts/pfam_dict.p","rb"))
pfam_defs=pickle.load(open("./databases/pickled_dicts/pfam_def.p","rb"))
cvp_dict=pickle.load(open("./databases/pickled_dicts/cvp_dict.p","rb"))

###Now parse through the BLAST files:

In [ ]:
from __future__ import division
from Bio.KEGG import REST
from pyfaidx import Fasta

In [140]:
#general info

#get number of genes in genome to know how many digits to use in locus tag


def get_digits(faa):
    faa=open(faa).read()
    digits=len(str(faa.count(">")))
    return digits

#create locus tag from protein sequence name in BLAST output file:
def get_locus_tag(line, digits, phage):
    query=line.split("\t")[0].split(" ")[0]
    number=query.split("_")[-1]
    z="0"*(digits-len(number))
    return "NVP"+phage.replace(".","")+"_"+z+number
        

from pyfaidx import Fasta

def get_prot_lens(faa_file, phage):
    len_dict={}
    digits=get_digits(faa_file)
    #def make_seq_len_dict(faa):
    f=Fasta(faa_file)
    for i in f.keys():
        name=get_locus_tag(i, digits=digits, phage=phage)
        length=len(str(f[i]))
        len_dict[name]=length
    return len_dict

#set up dict of general info from BLAST:
def set_up_blast_dict(blast, prod, faa):
    phage=blast.split("v")[0]
    digits=get_digits(faa)
    len_dict=get_prot_lens(faa, phage)
    records=[]
    blast_dict={}
    
    blast=open(blast).readlines()
    for line in blast:
        name=line.split(" ")[0]
        hit=line.split("\t")[1]
        lt=get_locus_tag(name, digits=digits, phage=phage)
        prot_len=len_dict[lt]
        aln_len=int(line.split("\t")[3])
        pct_id=float(line.split("\t")[2])
        ev=line.split("\t")[-2]
        pct_cov=(aln_len/prot_len)*100

        if pct_id>35 and pct_cov>75 and lt not in records:
            records.append(lt)
            blast_dict[lt]=[hit, pct_cov, pct_id, ev]
    return blast_dict



#functions for adding annotations/info to BLAST hit based on BLAST database

def add_kegg_descript(hit):
    desc= REST.kegg_find("genes", hit).read()
    K=re.search(r"K[0-9]{5}", desc)
    KEGG=K.group(0)
    a=re.search(r"(?<=K[0-9]{5}).*", desc)
    ann=a.group(0)
    return [KEGG, ann]

def add_cog_descript(hit):
    cog=cog_dict[(hit.split("|")[1])]
    func=cog_defs[cog].replace("\n","")
    return [cog, func]

def add_pfam_descript(hit):
    pfam=pfam_dict[hit].split(".")[0]
    function=pfam_defs[pfam].replace("\n","")
    return [pfam, function]

def add_aclame_descript(hit):
    annotation=aclame_dict[hit]
    return [hit, annotation]

def add_cvp_descript(hit):
    func=cvp_dict[hit]
    return [hit, func]

def add_tara_descript(hit):   #right now just adding the closest hit, TARA sequences come with COG/Pfam info etc 
    return [hit, hit]

db_dict={"kegg":add_kegg_descript, "cog":add_cog_descript, "pfam":add_pfam_descript, "aclame":add_aclame_descript,
        "cvp":add_cvp_descript, "tara":add_tara_descript}

In [135]:
##tests:
print add_kegg_descript('mtt:Ftrac_0033')==['K00287',' dihydrofolate reductase [EC:1.5.1.3]']
print add_cog_descript('gi|158522899|ref|YP_001530769.1|')==['COG3093','Plasmid maintenance system antidote protein VapI, contains XRE-type HTH domain']
print add_pfam_descript('F1D5B4_9CAUD/2-450')==['PF03237','Terminase-like family']
print add_aclame_descript('protein:plasmid:126053')==['protein:plasmid:126053','putative transcriptional regulator protein']
print add_cvp_descript('NCBI_PEP_323514092')==['NCBI_PEP_323514092','putative large terminase subunit ']
print add_tara_descript('OM-RGC.v1.003885702')==['OM-RGC.v1.003885702','OM-RGC.v1.003885702']



True
True
True
True
True
True


In [125]:
#add database specific annotations to BLAST dict
def annotated_blast_dict(blast, prod, faa, db):
    blast_dict=set_up_blast_dict(blast, prod, faa)
    blast_db_function=db_dict[db]
    for i in blast_dict.keys():
        hit=blast_dict[i][0]
        info=blast_db_function(hit)
        blast_dict[i]+=info
    
    return blast_dict

In [139]:
#testing functions
blast_dict=set_up_blast_dict("1.161.O.vs.Pfam.out","1.161.O.gene", "1.161.O.faa")
i=blast_dict.keys()[1]
hit=blast_dict[i][0]
blast_db_function=db_dict["pfam"]
info=blast_db_function(hit)
blast_dict[i]+=info
print blast_dict[i]

print annotated_blast_dict("1.161.O.vs.cogs_2003-2014.out","1.161.O.gene","1.161.O.faa", db="cog")

['F1D4J7_9CAUD/6-302', 99.34640522875817, 51.3, '4e-74', 'PF09615', 'CRISPR-associated protein (Cas_Csy3)']
{'NVP1161O_047': ['gi|383753766|ref|YP_005432669.1|', 102.35690235690235, 39.5, '2.2e-49', 'COG1518', 'CRISPR/Cas system-associated endonuclease Cas1'], 'NVP1161O_169': ['gi|387120486|ref|YP_006286369.1|', 101.0600706713781, 40.9, '4.3e-47', 'COG0207', 'Thymidylate synthase'], 'NVP1161O_168': ['gi|313674162|ref|YP_004052158.1|', 81.3953488372093, 35.7, '3.6e-12', 'COG0262', 'Dihydrofolate reductase'], 'NVP1161O_036': ['gi|158522899|ref|YP_001530769.1|', 101.04166666666667, 41.2, '1.1e-10', 'COG3093', 'Plasmid maintenance system antidote protein VapI, contains XRE-type HTH domain'], 'NVP1161O_208': ['gi|427711674|ref|YP_007060298.1|', 76.47058823529412, 37.4, '4.5e-06', 'COG0216', 'Protein chain release factor A']}


In [181]:
#i=prodigal line that begins with a location identifier..
#function meant to iterate over the length of a prodigal file every two lines starting at line 3 as such: 
'''
for i in range (2, len(open(prod_file).readlines())-1,2):
    get_prod_cds_info(i,...)
'''

def get_prod_cds_info(i, prod, digits, phage):  
    loc=prod[i]
    if len(loc.split())==2:
        if "complement" in prod[i].split()[1]:
            strand="-"
            start=loc.split("..")[1].replace(")\n","")
            stop=loc.split("(")[1].split("..")[0]
        else:
            strand="+"
            start=loc.split()[1].split("..")[0]
            stop=loc.split()[1].split("..")[1].replace("\n","")
        start=start.replace(">","").replace("<","")
        stop=stop.replace(">","").replace("<","")
    info=prod[i+1]
    number=info.split(";")[0].split("=")[2].split("_")[1]
    z="0"*(digits-len(number))
    t="NVP"+phage.replace(".","")+"_"+z+number
    return [t, start, stop, strand]

In [142]:
def find_best_hit(gene_id, dict_list):
    evals=1
    annotation=""
    best_hit=""
    for i in range(0, len(first_look)):
        if gene_id in dict_list[i].keys():
            hit=dict_list[i][gene_id]
            #print hit[1]
            if float(hit[3])<evals:
                evals=float(hit[3])
                best_hit=dict_names[i]
                annotation=hit[-1]
            #print(dict_names[i]+"\t"+hit[1]+"\t"+hit[-1])    
    #print("best annotation for"+gene_id+" is from "+best_hit+" with e-value "+str(evals)+" and annotation of "+annotation)
    return [annotation, best_hit]



#below: considers hits to more informative databases before less informative databases
#dict_list* are lists of blast_dicts and dl*_names are the names of the dicts in the same order

def find_best_hit2(gene_id, dict_list1, dl1_names, dict_list2=[], dl2_names=[]):
    evals=1
    annotation=""
    best_hit=""
    hits=[]
    es=[]
    names=[]
    for i in range(0, len(dict_list1)):
        if gene_id in dict_list1[i].keys():
            hit=dict_list1[i][gene_id]
            hits.append(hit[-1])
            es.append(float(hit[3]))
            names.append(dl1_names[i])
    if len(hits)>0:
        best_annotation=[hits[es.index(min(es))],names[es.index(min(es))]]
    else:
        for i in range(0, len(dict_list2)):
            if gene_id in dict_list2[i].keys():
                hit=dict_list2[i][gene_id]
                hits.append(hit[-1])
                es.append(float(hit[3]))
                names.append(dl2_names[i])
        if len(hits)>0:
            best_annotation=[hits[es.index(min(es))],names[es.index(min(es))]]
        else:
            best_annotation=["",""]

    #print("best annotation for"+gene_id+" is from "+best_hit+" with e-value "+str(evals)+" and annotation of "+annotation)
    return best_annotation 

In [143]:
def gff3_header(prod):
    Sequence=open(prod).readlines()[0].split(";")[2].split("=")[1].replace('"','')
    return Sequence+"\n"

In [176]:
#merge BLAST results into one gff3
    
def cds_blast_annotations_to_gff3(phage):
    #prodigal and fasta files:
    prod=phage+"gene"
    faa=phage+"faa"
    
    #blast files:
    pfam_blast=phage+"vs.Pfam.out"
    cog_blast=phage+"vs.cogs_2003-2014.out"
    aclame_blast=phage+"vs.aclame.out"
    cvp_blast=phage+"vs.CVP.out"
    kegg_blast=phage+"vs.kegg.out"
    tara_blast=phage+"vs.tara.out"

    Sequence=open(prod).readlines()[0].split(";")[2].split("=")[1].replace('"','')
    
    #set up dicts from all BLASTs
    kegg_blast_dict=annotated_blast_dict(blast=kegg_blast, prod=prod, faa=faa, db="kegg")
    pfam_blast_dict=annotated_blast_dict(blast=pfam_blast, prod=prod, faa=faa, db="pfam")
    cog_blast_dict=annotated_blast_dict(blast=cog_blast, prod=prod, faa=faa, db="cog")
    aclame_blast_dict=annotated_blast_dict(blast=aclame_blast, prod=prod, faa=faa, db="aclame")
    cvp_blast_dict=annotated_blast_dict(blast=cvp_blast, prod=prod, faa=faa, db="cvp")
    tara_blast_dict=annotated_blast_dict(blast=tara_blast, prod=prod, faa=faa, db="tara")

    #prioritize and name dicts:
    #preferred blast dbs to annotate from if there's a match:
    first_looks=[kegg_blast_dict, pfam_blast_dict, cog_blast_dict, aclame_blast_dict]
    flnames=["kegg","pfam","cog","aclame"]
    #secondary database(s) to annotate from:
    second_look=[cvp_blast_dict]
    slnames=["CVP"]
    #databases with orthologous groups to include in annotation
    OGs=[kegg_blast_dict, pfam_blast_dict, cog_blast_dict]
    OG_names=["KEGG","PFam","COG"]
    #databases where the closest hit will be referenced, but no other info will be provided:
    annotes=[aclame_blast_dict, cvp_blast_dict, tara_blast_dict]
    annotes_names=["ACLAME","CAMERA_viral_proteins","TARA_Oceans_Dataset"]
    
    out=""  #set up string to write to
    
    #run through annotations of each prodigal-identified CDS:
    prod=open(prod).readlines()
    digits=get_digits(faa)
    
    #write gff3 lines from prodigal files and blast dicts:
    for i in range(2,len(prod)-1,2):

        coords=get_prod_cds_info(i, prod, digits, phage)
        locus_tag=coords[0]
        start=coords[1]
        stop=coords[2]
        strand=coords[3]
        
        #set up col9
        col9="ID="+locus_tag
        
        #ID best hit:
        best_hits=find_best_hit2(locus_tag, dict_list1=first_looks, dl1_names=flnames, dict_list2=second_look, dl2_names=slnames)

        #establish name:
        Name=best_hits[0]
        if len(Name)==0:
            col9+='; Name=hypothetical protein'
        else:
            col9+="; Name="+Name.replace('"','')

        #Add OG annotations:
        for d in range(0, len(OGs)):
            og_dict=OGs[d]
            if locus_tag in og_dict.keys():
                col9+='; Ontology_term="'+OG_names[d]+":"+og_dict[locus_tag][-2]+'"'

        #Add db closest hits to notes
        for d in range(0, len(annotes)):
            annote_dict=annotes[d]
            if locus_tag in annote_dict.keys():
                col9+='; note="'+annotes_names[d]+"_best_match:"+annote_dict[locus_tag][-2]+'"'
        out+=Sequence+"\t"+"prod"+"\t"+"CDS"+"\t"+start+"\t"+stop+"\t"+"."+"\t"+strand+"\t"+"0"+"\t"+col9+"\n"

    return out

In [177]:
#test:
out=cds_blast_annotations_to_gff3("1.161.O.")
print str(out.split("\n")[0:10])

['Vibriophage_1.161.O._10N.261.48.C5\tprod\tCDS\t<3\t212\t.\t+\t0\tID=NVP1161O_001; Name=hypothetical protein', 'Vibriophage_1.161.O._10N.261.48.C5\tprod\tCDS\t388\t1881\t.\t+\t0\tID=NVP1161O_002; Name=Terminase-like family; Ontology_term="PFam:PF03237"; note="CAMERA_viral_proteins_best_match:NCBI_PEP_323514092"; note="TARA_Oceans_Dataset_best_match:OM-RGC.v1.003885702"', 'Vibriophage_1.161.O._10N.261.48.C5\tprod\tCDS\t2939\t1917\t.\t-\t0\tID=NVP1161O_003; Name=C-5 cytosine-specific DNA methylase; Ontology_term="PFam:PF00145"; note="CAMERA_viral_proteins_best_match:NCBI_PEP_535137"', 'Vibriophage_1.161.O._10N.261.48.C5\tprod\tCDS\t3368\t2985\t.\t-\t0\tID=NVP1161O_004; Name=hypothetical protein', 'Vibriophage_1.161.O._10N.261.48.C5\tprod\tCDS\t3787\t3380\t.\t-\t0\tID=NVP1161O_005; Name=predicted protein ; note="CAMERA_viral_proteins_best_match:CAMPEP_0000011774"; note="TARA_Oceans_Dataset_best_match:OM-RGC.v1.028578108"', 'Vibriophage_1.161.O._10N.261.48.C5\tprod\tCDS\t4106\t3735\t.\t-\

In [85]:
def CRISPR_gff3(input_fasta, crt_output):
    crtout=open(crt_output).readlines()

    sequence=open(input_fasta).readlines()
    name= [i.replace(">","") for i in sequence if i.startswith(">")][0]
    out=""
    for line in crtout:
        if line.startswith("CRISPR"):
            vec=line.split()
            number=vec[1]
            start=vec[3]
            stop=vec[5]
            ID="NVP"+name.split("_")[1].replace(".","")+"_CRISPR-like_"+number
            out+=name.replace("\n","")+"\t"+"crt"+"\tputative CRISPR feature\t%s\t%s\t.\t.\t.\tID=%s" % (start, stop, ID)
            out+=", note=CRISPR region\n"
    return out

In [88]:
def tRNA_scan_to_gff3(tRNAScanSE_file):
    if os.path.getsize(tRNAScanSE_file)>0:
        t=open(tRNAScanSE_file).readlines()
        tanns=""
        for line in t[3:]:
            l=line.split("\t")

            locus_tag="NVP"+l[0].split("_")[1].replace(".","")+"_tRNA_"+l[1]
            start=l[2]
            stop=l[3]
            if start<stop:
                strand="+"
            else:
                strand="-"
            aa=l[4]
            codon=l[5]
            SeqID=l[0]
            col9="ID="+locus_tag+", aa="+aa+", codon="+codon
            out=SeqID+"\t"+"tRNAScanSE"+"\t"+"tRNA"+"\t"+start+"\t"+stop+"\t"+l[-1].replace("\n","")+"\t"+strand+"\t"+"0"+"\t"+col9+"\n"
            tanns+=out
        return tanns
    else:
        print "no tRNAs found in genome"
        return ""

In [169]:
#put them all together:

def write_gff3_file(phage):
    prod=phage+".gene"
    faa=phage+".faa"
    genomic_fasta="./genomes/%sfinal.fasta" % phage
    
    out=open(phage+"test.gff3","w")
    #out.write(gff3_header(phage+"gene"))
    out.write(cds_blast_annotations_to_gff3(phage))
    
    trna="../tRNA_info/data/nahant_tRNA_count/%strnas.txt" % phage 
    if os.path.getsize(trna)>0:
        out.write(tRNA_scan_to_gff3(trna))
    
    crt_output=phage+"crt"
    out.write(CRISPR_gff3(genomic_fasta, crt_output))
    
    
    out.close()


In [182]:
write_gff3_file("1.161.O.")

In [11]:
# %load ./scripts/annotation_functions.py
#Functions to run annotation-associated programs:

import subprocess
import os
import cPickle as pickle
from __future__ import division
from Bio.KEGG import REST
from pyfaidx import Fasta

def run_prodigal_phage(inputfasta, out_gene, out_prot):
    to_run="prodigal -i "+inputfasta+" -o "+out_gene+" -a "+out_prot+" -p meta"
    subprocess.call(to_run.split(" "))
    
def run_formatudb(fastafile, databasefile="db.udb", ublast_path="/home/sbiller/usearch7.0.1090_i86linux64"):
    to_run=ublast_path+" -makeudb_ublast "+fastafile+" -output "+databasefile
    subprocess.call(to_run.split(" "))
    
def run_ublastp(fastafile, out_file, udb, evalue, ublast_path="/home/sbiller/usearch7.0.1090_i86linux64"):
    to_run=ublast_path+" -ublast "+fastafile+" -db "+udb+" -evalue "+evalue+" -accel 0.5 -blast6out "+out_file+" -top_hit_only"
    subprocess.call(to_run.split(" "))
    
def run_trna_scan(input_file, output):
    if os.path.exists(output):
        os.remove(output)
    args=["tRNAscan-SE", "-o", output, "-G", "-D","-N", input_file]
    subprocess.call(args)
    print("tRNA scan of "+input_file+" is done!")

def run_crt(path_to_crt, input_fasta, output):
    args="java -cp "+path_to_crt+" crt -minNR 2 "+input_fasta+" "+output
    subprocess.call(args.split(" "))
    
#general info

#get number of genes in genome to know how many digits to use in locus tag


def get_digits(faa):
    faa=open(faa).read()
    digits=len(str(faa.count(">")))
    return digits

#create locus tag from protein sequence name in BLAST output file:
def get_locus_tag(line, digits, phage):
    query=line.split("\t")[0].split(" ")[0]
    number=query.split("_")[-1]
    z="0"*(digits-len(number))
    return "NVP"+phage.replace(".","")+"_"+z+number
        

from pyfaidx import Fasta

def get_prot_lens(faa_file, phage):
    len_dict={}
    digits=get_digits(faa_file)
    #def make_seq_len_dict(faa):
    f=Fasta(faa_file)
    for i in f.keys():
        name=get_locus_tag(i, digits=digits, phage=phage)
        length=len(str(f[i]))
        len_dict[name]=length
    return len_dict

#set up dict of general info from BLAST:
def set_up_blast_dict(blast, prod, faa, phage):
    digits=get_digits(faa)
    len_dict=get_prot_lens(faa, phage)
    records=[]
    blast_dict={}
    
    blast=open(blast).readlines()
    for line in blast:
        name=line.split(" ")[0]
        hit=line.split("\t")[1]
        lt=get_locus_tag(name, digits=digits, phage=phage)
        prot_len=len_dict[lt]
        aln_len=int(line.split("\t")[3])
        pct_id=float(line.split("\t")[2])
        ev=line.split("\t")[-2]
        pct_cov=(aln_len/prot_len)*100

        if pct_id>35 and pct_cov>75 and lt not in records:
            records.append(lt)
            blast_dict[lt]=[hit, pct_cov, pct_id, ev]
    return blast_dict

#load blast database dictionaries:
if os.path.exists("/nobackup1/jbrown/annotation/databases/pickled_dicts/"):
    aclame_dict=pickle.load(open("/nobackup1/jbrown/annotation/databases/pickled_dicts/aclame_dict.p","rb"))
    cog_dict=pickle.load(open("/nobackup1/jbrown/annotation/databases/pickled_dicts/cog_dict.p","rb"))
    cog_defs=pickle.load(open("/nobackup1/jbrown/annotation/databases/pickled_dicts/cog_def.p","rb"))
    pfam_dict=pickle.load(open("/nobackup1/jbrown/annotation/databases/pickled_dicts/pfam_dict.p","rb"))
    pfam_defs=pickle.load(open("/nobackup1/jbrown/annotation/databases/pickled_dicts/pfam_def.p","rb"))
    cvp_dict=pickle.load(open("/nobackup1/jbrown/annotation/databases/pickled_dicts/cvp_dict.p","rb"))
    dict_check=True
elif os.path.exists("./databases/pickled_dicts/"):
    aclame_dict=pickle.load(open("./databases/pickled_dicts/aclame_dict.p","rb"))
    cog_dict=pickle.load(open("./databases/pickled_dicts/cog_dict.p","rb"))
    cog_defs=pickle.load(open("./databases/pickled_dicts/cog_def.p","rb"))
    pfam_dict=pickle.load(open("./databases/pickled_dicts/pfam_dict.p","rb"))
    pfam_defs=pickle.load(open("./databases/pickled_dicts/pfam_def.p","rb"))
    cvp_dict=pickle.load(open("./databases/pickled_dicts/cvp_dict.p","rb"))
    dict_check=True
else:
    print("blast database dictionaries not found")
    dict_check=False

#functions for adding annotations/info to BLAST hit based on BLAST database

def add_kegg_descript(hit):
    
    desc= REST.kegg_find("genes", hit).read()
    K=re.search(r"K[0-9]{5}", desc)
    KEGG=K.group(0)
    a=re.search(r"(?<=K[0-9]{5}).*", desc)
    ann=a.group(0)
    return [KEGG, ann]

def add_cog_descript(hit):
    if dict_check:
        cog=cog_dict[(hit.split("|")[1])]
        func=cog_defs[cog].replace("\n","")
        return [cog, func]
    else:
        print "COG database is not loaded"
        return ""

def add_pfam_descript(hit):
    if dict_check:
        pfam=pfam_dict[hit].split(".")[0]
        function=pfam_defs[pfam].replace("\n","")
        return [pfam, function]
    else:
        print "pfam database is not loaded"
        return ""

def add_aclame_descript(hit):
    if dict_check:
        annotation=aclame_dict[hit]
        return [hit, annotation]
    else:
        print "aclame database not loaded"
        return ""

def add_cvp_descript(hit):
    func=cvp_dict[hit]
    return [hit, func]

def add_tara_descript(hit):   #right now just adding the closest hit, TARA sequences come with COG/Pfam info etc 
    return [hit, hit]

db_dict={"kegg":add_kegg_descript, "cog":add_cog_descript, "pfam":add_pfam_descript, "aclame":add_aclame_descript,
        "cvp":add_cvp_descript, "tara":add_tara_descript}

def annotated_blast_dict(blast, prod, faa, db, phage):
    blast_dict=set_up_blast_dict(blast, prod, faa, phage)
    blast_db_function=db_dict[db]
    for i in blast_dict.keys():
        hit=blast_dict[i][0]
        info=blast_db_function(hit)
        blast_dict[i]+=info
    
    return blast_dict

#i=prodigal line that begins with a location identifier..
#function meant to iterate over the length of a prodigal file every two lines starting at line 3 as such: 
'''
for i in range (2, len(open(prod_file).readlines())-1,2):
    get_prod_cds_info(i,...)
'''

def get_prod_cds_info(i, prod, digits, phage):  
    loc=prod[i]
    if len(loc.split())==2:
        if "complement" in prod[i].split()[1]:
            strand="-"
            start=loc.split("..")[1].replace(")\n","")
            stop=loc.split("(")[1].split("..")[0]
        else:
            strand="+"
            start=loc.split()[1].split("..")[0]
            stop=loc.split()[1].split("..")[1].replace("\n","")
        start=start.replace(">","").replace("<","")
        stop=stop.replace(">","").replace("<","")
    info=prod[i+1]
    number=info.split(";")[0].split("=")[2].split("_")[1]
    z="0"*(digits-len(number))
    t="NVP"+phage.replace(".","")+"_"+z+number
    return [t, start, stop, strand]

#below: considers hits to more informative databases before less informative databases
#dict_list* are lists of blast_dicts and dl*_names are the names of the dicts in the same order

def find_best_hit2(gene_id, dict_list1, dl1_names, dict_list2=[], dl2_names=[]):
    evals=1
    annotation=""
    best_hit=""
    hits=[]
    es=[]
    names=[]
    for i in range(0, len(dict_list1)):
        if gene_id in dict_list1[i].keys():
            hit=dict_list1[i][gene_id]
            hits.append(hit[-1])
            es.append(float(hit[3]))
            names.append(dl1_names[i])
    if len(hits)>0:
        best_annotation=[hits[es.index(min(es))],names[es.index(min(es))]]
    else:
        for i in range(0, len(dict_list2)):
            if gene_id in dict_list2[i].keys():
                hit=dict_list2[i][gene_id]
                hits.append(hit[-1])
                es.append(float(hit[3]))
                names.append(dl2_names[i])
        if len(hits)>0:
            best_annotation=[hits[es.index(min(es))],names[es.index(min(es))]]
        else:
            best_annotation=["",""]

    #print("best annotation for"+gene_id+" is from "+best_hit+" with e-value "+str(evals)+" and annotation of "+annotation)
    return best_annotation 

def gff3_header(prod):
    Sequence=open(prod).readlines()[0].split(";")[2].split("=")[1].replace('"','')
    return Sequence+"\n"

#merge BLAST results into one gff3
    
def cds_blast_annotations_to_gff3(phage):
    #prodigal and fasta files:
    prod=phage+"gene"
    faa=phage+"faa"
    
    #blast files:
    pfam_blast=phage+"vs.Pfam.out"
    cog_blast=phage+"vs.cogs_2003-2014.out"
    aclame_blast=phage+"vs.aclame.out"
    cvp_blast=phage+"vs.CVP.out"
    kegg_blast=phage+"vs.kegg.out"
    tara_blast=phage+"vs.tara.out"

    Sequence=open(prod).readlines()[0].split(";")[2].split("=")[1].replace('"','')
    
    #set up dicts from all BLASTs
    kegg_blast_dict=annotated_blast_dict(blast=kegg_blast, prod=prod, faa=faa, db="kegg", phage=phage)
    pfam_blast_dict=annotated_blast_dict(blast=pfam_blast, prod=prod, faa=faa, db="pfam", phage=phage)
    cog_blast_dict=annotated_blast_dict(blast=cog_blast, prod=prod, faa=faa, db="cog", phage=phage)
    aclame_blast_dict=annotated_blast_dict(blast=aclame_blast, prod=prod, faa=faa, db="aclame", phage=phage)
    cvp_blast_dict=annotated_blast_dict(blast=cvp_blast, prod=prod, faa=faa, db="cvp", phage=phage)
    tara_blast_dict=annotated_blast_dict(blast=tara_blast, prod=prod, faa=faa, db="tara", phage=phage)

    #prioritize and name dicts:
    #preferred blast dbs to annotate from if there's a match:
    first_looks=[kegg_blast_dict, pfam_blast_dict, cog_blast_dict, aclame_blast_dict]
    flnames=["kegg","pfam","cog","aclame"]
    #secondary database(s) to annotate from:
    second_look=[cvp_blast_dict]
    slnames=["CVP"]
    #databases with orthologous groups to include in annotation
    OGs=[kegg_blast_dict, pfam_blast_dict, cog_blast_dict]
    OG_names=["KEGG","PFam","COG"]
    #databases where the closest hit will be referenced, but no other info will be provided:
    annotes=[aclame_blast_dict, cvp_blast_dict, tara_blast_dict]
    annotes_names=["ACLAME","CAMERA_viral_proteins","TARA_Oceans_Dataset"]
    
    out=""  #set up string to write to
    
    #run through annotations of each prodigal-identified CDS:
    prod=open(prod).readlines()
    digits=get_digits(faa)
    
    #write gff3 lines from prodigal files and blast dicts:
    for i in range(2,len(prod)-1,2):

        coords=get_prod_cds_info(i, prod, digits, phage)
        locus_tag=coords[0]
        start=coords[1]
        stop=coords[2]
        strand=coords[3]
        
        #set up col9
        col9="ID="+locus_tag
        
        #ID best hit:
        best_hits=find_best_hit2(locus_tag, dict_list1=first_looks, dl1_names=flnames, dict_list2=second_look, dl2_names=slnames)

        #establish name:
        Name=best_hits[0]
        if len(Name)==0:
            col9+='; Name=hypothetical protein'
        else:
            col9+="; Name="+Name.replace('"','')

        #Add OG annotations:
        for d in range(0, len(OGs)):
            og_dict=OGs[d]
            if locus_tag in og_dict.keys():
                col9+='; Ontology_term="'+OG_names[d]+":"+og_dict[locus_tag][-2]+'"'

        #Add db closest hits to notes
        for d in range(0, len(annotes)):
            annote_dict=annotes[d]
            if locus_tag in annote_dict.keys():
                col9+='; note="'+annotes_names[d]+"_best_match:"+annote_dict[locus_tag][-2]+'"'
        out+=Sequence+"\t"+"prod"+"\t"+"CDS"+"\t"+start+"\t"+stop+"\t"+"."+"\t"+strand+"\t"+"0"+"\t"+col9+"\n"

    return out

def CRISPR_gff3(input_fasta, crt_output):
    crtout=open(crt_output).readlines()

    sequence=open(input_fasta).readlines()
    name= [i.replace(">","") for i in sequence if i.startswith(">")][0]
    out=""
    for line in crtout:
        if line.startswith("CRISPR"):
            vec=line.split()
            number=vec[1]
            start=vec[3]
            stop=vec[5]
            ID="NVP"+name.split("_")[1].replace(".","")+"_CRISPR-like_"+number
            out+=name.replace("\n","")+"\t"+"crt"+"\tputative CRISPR feature\t%s\t%s\t.\t.\t.\tID=%s" % (start, stop, ID)
            out+=", note=CRISPR region\n"
    return out

def tRNA_scan_to_gff3(tRNAScanSE_file):
    if os.path.getsize(tRNAScanSE_file)>0:
        t=open(tRNAScanSE_file).readlines()
        tanns=""
        for line in t[3:]:
            l=line.split("\t")

            locus_tag="NVP"+l[0].split("_")[1].replace(".","")+"_tRNA_"+l[1]
            start=l[2]
            stop=l[3]
            if start<stop:
                strand="+"
            else:
                strand="-"
            aa=l[4]
            codon=l[5]
            SeqID=l[0]
            col9="ID="+locus_tag+", aa="+aa+", codon="+codon
            out=SeqID+"\t"+"tRNAScanSE"+"\t"+"tRNA"+"\t"+start+"\t"+stop+"\t"+l[-1].replace("\n","")+"\t"+strand+"\t"+"0"+"\t"+col9+"\n"
            tanns+=out
        return tanns
    else:
        print "no tRNAs found in genome"
        return ""
    
#put them all together:
def write_gff3_file(phage):
    prod=phage+".gene"
    faa=phage+".faa"
    genomic_fasta="./genomes/%sfinal.fasta" % phage
    
    out=open(phage+"test.gff3","w")
    #out.write(gff3_header(phage+"gene"))
    out.write(cds_blast_annotations_to_gff3(phage))
    
    trna="../tRNA_info/data/nahant_tRNA_count/%strnas.txt" % phage 
    if os.path.getsize(trna)>0:
        out.write(tRNA_scan_to_gff3(trna))
    
    crt_output=phage+"crt"
    out.write(CRISPR_gff3(genomic_fasta, crt_output))
    
    out.close()

def query_tara_db(tid):
    conn=sqlite3.connect('/pool001/jbrown/tara_db.sqlite')
    c=conn.cursor()

    c.execute("SELECT * from taratbl where ID='"+tid+"'")
    output=c.fetchall()
    gene=output[0][1]
    egg=output[0][2]
    ko=output[0][3]
    kfunc=output[0][4]
    conn.close()
    
    return [gene, egg, ko, kfunc]

In [14]:
import re

BLAST annotations to table:

In [16]:
def cds_blast_annotations_to_table(phage):
    #prodigal and fasta files:
    prod=phage+"gene"
    faa=phage+"faa"
    
    #blast files:
    pfam_blast=phage+"vs.Pfam.out"
    cog_blast=phage+"vs.cogs_2003-2014.out"
    aclame_blast=phage+"vs.aclame.out"
    cvp_blast=phage+"vs.CVP.out"
    kegg_blast=phage+"vs.kegg.out"
    tara_blast=phage+"vs.tara.translated.out"

    Sequence=open(prod).readlines()[0].split(";")[2].split("=")[1].replace('"','')
    
    #set up dicts from all BLASTs
    kegg_blast_dict=annotated_blast_dict(blast=kegg_blast, prod=prod, faa=faa, db="kegg", phage=phage)
    pfam_blast_dict=annotated_blast_dict(blast=pfam_blast, prod=prod, faa=faa, db="pfam", phage=phage)
    cog_blast_dict=annotated_blast_dict(blast=cog_blast, prod=prod, faa=faa, db="cog", phage=phage)
    aclame_blast_dict=annotated_blast_dict(blast=aclame_blast, prod=prod, faa=faa, db="aclame", phage=phage)
    cvp_blast_dict=annotated_blast_dict(blast=cvp_blast, prod=prod, faa=faa, db="cvp", phage=phage)
    tara_blast_dict=annotated_blast_dict(blast=tara_blast, prod=prod, faa=faa, db="tara", phage=phage)

    #prioritize and name dicts:
    #preferred blast dbs to annotate from if there's a match:
    first_looks=[kegg_blast_dict, pfam_blast_dict, cog_blast_dict, aclame_blast_dict]
    flnames=["kegg","pfam","cog","aclame"]
    #secondary database(s) to annotate from:
    second_look=[cvp_blast_dict]
    slnames=["CVP"]
    #databases with orthologous groups to include in annotation
    OGs=[kegg_blast_dict, pfam_blast_dict, cog_blast_dict]
    OG_names=["KEGG","PFam","COG"]
    #databases where the closest hit will be referenced, but no other info will be provided:
    annotes=[aclame_blast_dict, cvp_blast_dict, tara_blast_dict]
    annotes_names=["ACLAME","CAMERA_viral_proteins","TARA_Oceans_Dataset"]
    
    out=open(phage+"annotations.txt","w")
    out.write("Genome\tlocus_tag\ttype\tstart\tstop\tstrand\tbest_hit_annotation\t")
    out.write("KEGG\tPFam\tCOG\tACLAME\tCVP\tTARA\n")
    
    #run through annotations of each prodigal-identified CDS:
    prod=open(prod).readlines()
    digits=get_digits(faa)
    
    #write gff3 lines from prodigal files and blast dicts:
    for i in range(2,len(prod)-1,2):
        out.write(Sequence+"\t")
        
        
        coords=get_prod_cds_info(i, prod, digits, phage)
        locus_tag=coords[0]
        start=coords[1]
        stop=coords[2]
        strand=coords[3]
        
        out.write(locus_tag+"\tcds\t"+start+"\t"+stop+"\t"+strand+"\t")
        
        #ID best hit:
        best_hits=find_best_hit2(locus_tag, dict_list1=first_looks, dl1_names=flnames, dict_list2=second_look, dl2_names=slnames)

        #establish name:
        Annotation=best_hits[0]
        if len(Annotation)==0:
            Annotation='hypothetical protein'
        else:
            Annotation=Annotation.replace('"','')
        out.write(Annotation+"\t")
        
        #Add OG annotations:
        for d in range(0, len(OGs)):
            og_dict=OGs[d]
            if locus_tag in og_dict.keys():
                out.write(og_dict[locus_tag][-2]+"\t")
            else:
                out.write(" \t")

        #Add db closest hits to notes
        for d in range(0, len(annotes)):
            annote_dict=annotes[d]
            if locus_tag in annote_dict.keys():
                out.write(annote_dict[locus_tag][-2]+'\t')
            else:
                out.write(" \t")
        out.write("\n")

    out.close()

In [17]:
cds_blast_annotations_to_table("1.161.O.")

In [ ]:
#paths
prod_path="/nobackup1/jbrown/annotation/gene/"
faa_path="/nobackup1/jbrown/annotation/proteins/"
    
pfam_blast_path="/nobackup1/jbrown/annotation/blasts/Pfam/"
cog_blast_path="/nobackup1/jbrown/annotation/blasts/cogs_2003-2014/"
aclame_blast_path="/nobackup1/jbrown/annotation/blasts/aclame/"
cvp_blast_path="/nobackup1/jbrown/annotation/blasts/CVP/"
kegg_blast_path="/nobackup1/jbrown/annotation/blasts/kegg/"
tara_blast_path="/nobackup1/jbrown/annotation/blasts/tara.translated/"